In [13]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt



import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from skimage import color, exposure
from sklearn.metrics import accuracy_score

%load_ext tensorboard

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/road_sign_prediction'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/road_sign_prediction


In [0]:
#load data
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
#these operation will transform our y dasets to categorical meaning that for example y_train = 5 will be transformed
#into a 0,1 vector with 43 values when 1 will be inputed in the coressponding number
if y_train.ndim == 1 : y_train = to_categorical(y_train)
if y_test.ndim == 1 : y_test = to_categorical(y_test)

In [0]:
#input shape i 32x32 pixels with 3 channels (RGB)
input_shape = X_train.shape[1:]
#number of classes i.e. road signs is 43
num_classes = y_train.shape[1]

In [0]:
#function from 4th notebook in github project

def train_model(model, X_train, y_train, params_fit={}):
  
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')

  #log directory 
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  #pass to log
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq= 1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return model


def predict(model_trained, X_test, y_test, scoring = accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis = 1)

  return scoring(y_test_norm, y_pred)


def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout (0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024,activation='relu'),
  Dropout(0.3),

  Dense(1024,activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')
])

In [9]:
#remember to change to GPU if possible! (colab: runtime --> change runtime typ --> GPU. You will have to run code agian)
train_and_predict( get_cnn_v5(input_shape, num_classes) )

Epoch 1/5
272/272 [==============================] - 13s 48ms/step - loss: 2.5209 - accuracy: 0.2966 - val_loss: 1.1083 - val_accuracy: 0.6296
Epoch 2/5
272/272 [==============================] - 12s 45ms/step - loss: 0.8769 - accuracy: 0.7177 - val_loss: 0.2339 - val_accuracy: 0.9285
Epoch 3/5
272/272 [==============================] - 12s 45ms/step - loss: 0.4000 - accuracy: 0.8735 - val_loss: 0.0873 - val_accuracy: 0.9740
Epoch 4/5
272/272 [==============================] - 12s 46ms/step - loss: 0.2458 - accuracy: 0.9263 - val_loss: 0.0578 - val_accuracy: 0.9823
Epoch 5/5
272/272 [==============================] - 12s 45ms/step - loss: 0.1839 - accuracy: 0.9456 - val_loss: 0.0473 - val_accuracy: 0.9868


0.9634920634920635

In [0]:
#lets make the gen_cnn_v5 function more general

def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout (params['dropout_cnn_block_one']),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_two']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_three']),

  Flatten(),

  Dense(1024,activation='relu'),
  Dropout(params['dropout_dense_block_one']),

  Dense(1024,activation='relu'),
  Dropout(params['dropout_dense_block_two']),

  Dense(num_classes, activation='softmax')
])

In [0]:
#now time for hyperopt - we will define the space where it will search for the best parameters

def fun_obj(params):

  model = get_model(params)
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')

  model.fit(
      X_train,
      y_train,
      batch_size=128, #int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0
      )
  
  score =  model.evaluate(X_test, y_test, verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  #minus to minimalize
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}



In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one' : hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two' : hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three' : hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one' : hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two' : hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

In [41]:
best = fmin(
    fun_obj,
    space,
    tpe.suggest,
    25, #max evals
    Trials() #where to save
)

{'batch_size': 190.0, 'dropout_cnn_block_one': 0.3665595652608205, 'dropout_cnn_block_three': 0.386798415000611, 'dropout_cnn_block_two': 0.4230526418001501, 'dropout_dense_block_one': 0.4774159492259453, 'dropout_dense_block_two': 0.4677168323297812}
accuracy=0.940816342830658
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.4097294177854268, 'dropout_cnn_block_three': 0.36729237054949165, 'dropout_cnn_block_two': 0.33451426270453904, 'dropout_dense_block_one': 0.6954654266361721, 'dropout_dense_block_two': 0.43104453532729614}
accuracy=0.9428571462631226
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.4352569433405905, 'dropout_cnn_block_three': 0.4497448149834924, 'dropout_cnn_block_two': 0.49953766991436444, 'dropout_dense_block_one': 0.5531381512742051, 'dropout_dense_block_two': 0.5622755050448494}
accuracy=0.8730158805847168
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.3161265657484704, 'dropout_cnn_block_three': 0.4962620373069179, 'dropout_cnn_block_two': 0.34839684416728

In [0]:
!git config --global user.email "kuband93@gmail.com"
!git config --global user.name "Jakub Andrzejewski"

In [3]:
!git add 5_tunig_hyperparameters.ipynb

fatal: not a git repository (or any of the parent directories): .git


In [0]:
!